In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("food_sales.csv", encoding="utf-8-sig")
weather = pd.read_csv("df_weather.csv", encoding="utf-8-sig")
hol = pd.read_csv("df_holiday.csv", encoding="utf-8-sig")




In [2]:
df["日期"] = pd.to_datetime(df["日期"])
weather["ds"] = pd.to_datetime(weather["ds"])
hol["date"] = pd.to_datetime(hol["date"])

In [3]:
df.head()

,日期,菜品,销量
0,2025-05-01,掌中宝,66.0
1,2025-05-01,烤羊肉串,168.0
2,2025-05-01,老坛酸菜味江团,3.0
3,2025-05-01,老坛酸菜味海鲈鱼,3.0
4,2025-05-01,老坛酸菜味黔鱼,3.0


### 数据准备 （ds/y + 天气 + holidays）

In [4]:
#选菜品+对可能的日期做预定义
dish_name = "麻辣味黔鱼"
df_one = df.loc[df["菜品"] == dish_name, ["日期", "销量"]].copy()

df_prophet = (
    df_one.groupby("日期", as_index=False)["销量"].sum()
    .rename(columns={"日期": "ds", "销量": "y"})
    .sort_values("ds")
)

# 缺失日期怎么定义：这里假设缺失=0（若缺失=未知，就不要 fillna(0)）
full_ds = pd.date_range(df_prophet["ds"].min(), df_prophet["ds"].max(), freq="D")
df_prophet = (
    df_prophet.set_index("ds").reindex(full_ds).rename_axis("ds").reset_index()
)
df_prophet["y"] = df_prophet["y"].fillna(0.0)


In [5]:
df_prophet.head()

,ds,y
0,2025-05-01,41.0
1,2025-05-02,44.0
2,2025-05-03,37.0
3,2025-05-04,44.0
4,2025-05-05,42.0


In [ ]:

# holidays 从 df_holiday.csv 来
hol.columns = hol.columns.str.replace("\ufeff", "", regex=False)
hol["date"] = pd.to_datetime(hol["date"])
holidays = (
    hol.rename(columns={"date": "ds", "holiday_name": "holiday"})[
        ["ds", "holiday", "lower_window", "upper_window"]
    ]
    .copy()
)
holidays = holidays[holidays["holiday"].fillna("").ne("")]

#合并天气（Prophet/XGB 都会用到）
reg_cols = ["rain_mm", "avg_temp_c", "avg_humidity_pct"]
df_all = df_prophet.merge(weather[["ds"] + reg_cols], on="ds", how="left").sort_values("ds")

# 缺失处理（关键：Prophet 的 regressor 不能有 NaN；XGB 也尽量不要）
for c in reg_cols:
    df_all[c] = df_all[c].ffill()
    df_all[c] = df_all[c].fillna(df_all[c].mean())


### 2）Prophet 作为“特征生成器”

In [7]:
from prophet import Prophet

def fit_prophet_feature_generator(train_df: pd.DataFrame, holidays: pd.DataFrame):
    """
    Prophet 在这里不直接当最终预测器，只负责把时间结构/外生回归的“解释”变成特征。
    """
    m = Prophet(
        weekly_seasonality=True,
        yearly_seasonality=False,
        daily_seasonality=False,
        holidays=holidays,
    )
    # 外生回归量：让 Prophet 把天气影响“编码”进它的分解结果里
    for c in reg_cols:
        m.add_regressor(c)

    m.fit(train_df[["ds", "y"] + reg_cols])
    return m


def make_xgb_features_from_prophet(m: Prophet, df_for_predict: pd.DataFrame) -> pd.DataFrame:
    """
    输入 df_for_predict 必须包含 ds + regressor 列（否则 Prophet predict 会缺列）。
    输出：XGBoost 可用的特征表（含 Prophet 生成的结构特征 + 日期特征 + 原始天气）。
    """
    fcst = m.predict(df_for_predict[["ds"] + reg_cols])

    feat = pd.DataFrame({"ds": fcst["ds"]})

    # ===== Prophet 生成的“时间结构特征” =====
    # yhat 不作为最终输出，只作为 XGB 的一个特征（这就是 feature stacking 的核心）
    for c in ["yhat", "trend", "weekly", "holidays", "additive_terms", "multiplicative_terms", "extra_regressors_additive"]:
        if c in fcst.columns:
            feat[c] = fcst[c]

    # ===== 并行加入“原始外生变量” =====
    # 让 XGB 自己决定怎么用天气；同时 Prophet 的 extra_regressors_additive 提供“结构化解释”
    for c in reg_cols:
        feat[c] = df_for_predict[c].to_numpy()

    # ===== 日期类特征（给 XGB 一些非参数化自由度）=====
    ds = pd.to_datetime(feat["ds"])
    feat["dow"] = ds.dt.dayofweek
    feat["month"] = ds.dt.month
    feat["is_weekend"] = (feat["dow"] >= 5).astype(int)

    return feat


### 3）用“滚动/扩展窗口”生成 OOF 特征（避免泄漏）

In [8]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

horizon = 14
n = len(df_all)

# 你可以调大 folds，让 XGB 看到更多“无泄漏”的样本
folds = 6
min_train_size = 60  # 至少先给 Prophet 一段历史，不然很不稳

oof_rows = []

# 从后往前取 folds 个验证窗口，每个窗口长度 = horizon
for i in range(folds, 0, -1):
    val_end = n - (i - 1) * horizon
    val_start = val_end - horizon
    train_end = val_start

    if train_end < min_train_size:
        continue

    train_df = df_all.iloc[:train_end].copy()
    val_df = df_all.iloc[val_start:val_end].copy()

    # 关键：Prophet 只用 train_df 拟合，然后只对 val_df 生成特征
    m = fit_prophet_feature_generator(train_df, holidays)
    X_val = make_xgb_features_from_prophet(m, val_df)

    y_val = val_df[["ds", "y"]].copy()
    fold_df = X_val.merge(y_val, on="ds", how="left")

    oof_rows.append(fold_df)

oof = pd.concat(oof_rows, ignore_index=True).sort_values("ds")

feature_cols = [c for c in oof.columns if c not in ["ds", "y"]]
X_train = oof[feature_cols]
y_train = oof["y"]

xgb = XGBRegressor(
    n_estimators=800,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42,
)
xgb.fit(X_train, y_train)

# 这只是“OOF窗口”上的训练误差（更像 sanity check），完整评估可再做严格 walk-forward
pred_oof = xgb.predict(X_train)
print("OOF-window MAE:", mean_absolute_error(y_train, pred_oof))


00:46:26 - cmdstanpy - INFO - Chain [1] start processing
00:46:26 - cmdstanpy - INFO - Chain [1] done processing
00:46:27 - cmdstanpy - INFO - Chain [1] start processing
00:46:27 - cmdstanpy - INFO - Chain [1] done processing
00:46:27 - cmdstanpy - INFO - Chain [1] start processing
00:46:27 - cmdstanpy - INFO - Chain [1] done processing
00:46:27 - cmdstanpy - INFO - Chain [1] start processing
00:46:27 - cmdstanpy - INFO - Chain [1] done processing
00:46:27 - cmdstanpy - INFO - Chain [1] start processing
00:46:27 - cmdstanpy - INFO - Chain [1] done processing
00:46:27 - cmdstanpy - INFO - Chain [1] start processing
00:46:27 - cmdstanpy - INFO - Chain [1] done processing


OOF-window MAE: 0.0005269277663457961


### 4) 用“全量 Prophet”生成未来特征，然后 XGB 输出最终预测

In [9]:
# 1) 训练最终 Prophet 特征生成器（用全量历史）
m_full = fit_prophet_feature_generator(df_all, holidays)

# 2) 构造未来 ds
future_ds = pd.date_range(df_all["ds"].max() + pd.Timedelta(days=1), periods=horizon, freq="D")
future_df = pd.DataFrame({"ds": future_ds})

# 关键：未来天气必须补齐（你说后续用 API/MySQL，这里就从 weather_future 来）
# 先用一个占位：用最后一天的天气做“平推”，保证代码可跑通（上线务必替换）
last_weather = df_all.iloc[-1][reg_cols]
for c in reg_cols:
    future_df[c] = float(last_weather[c])

# 3) 生成未来特征 -> XGB 输出最终预测
X_future = make_xgb_features_from_prophet(m_full, future_df)
y_future_hat = xgb.predict(X_future[feature_cols])

pred_future = future_df[["ds"]].copy()
pred_future["yhat_xgb"] = y_future_hat
pred_future.head()


00:46:29 - cmdstanpy - INFO - Chain [1] start processing
00:46:29 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat_xgb
0,2026-01-01,23.041216
1,2026-01-02,25.184177
2,2026-01-03,25.150871
3,2026-01-04,24.184271
4,2026-01-05,22.866838
